<a href="https://colab.research.google.com/github/tonyphan132/COMP352-A2/blob/master/COMP_333_Project_NYC_Car_Crashes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#Importing libraries
import pandas as pd
import numpy as np
import sqlite3
from google.colab import drive
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="nyc_borough_geocoder")
drive.mount("/content/drive")
common_folder_path = '/content/drive/My Drive/COMP-333-Project/'

dataFrame = pd.read_csv(common_folder_path + "Crashes.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-14-20e56763e5db>:11: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dataFrame = pd.read_csv(common_folder_path + "Crashes.csv")


In [ ]:
print(len(dataFrame))

2127006


In [12]:

count_manhattan = len(dataFrame[dataFrame["BOROUGH"] == "MANHATTAN"])
count_brooklyn = len(dataFrame[dataFrame["BOROUGH"] == "BROOKLYN"])
count_queens = len(dataFrame[dataFrame["BOROUGH"] == "QUEENS"])
count_bronx = len(dataFrame[dataFrame["BOROUGH"] == "BRONX"])
count_statenisland = len(dataFrame[dataFrame["BOROUGH"] == "STATEN ISLAND"])
nan_count = len(dataFrame[dataFrame["BOROUGH"].isna()])

print(count_manhattan + count_brooklyn + count_queens + count_bronx + count_statenisland + nan_count - len(dataFrame))


0


In [19]:
def geocode_borough(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), language='en', exactly_one=True)
        if location:
            # Extract the borough from the address
            address = location.raw.get('address', {})
            borough = address.get('borough', None)
            if not borough:  # If borough is None, try to use the suburb or neighbourhood
                borough = address.get('suburb', address.get('neighbourhood', None))
            return borough
        else:
            return None
    except Exception as e:
        print(f"Error geocoding ({lat}, {lon}): {e}")
        return None



# Show the results
# print(test_df)

In [22]:
filtered_df = dataFrame.dropna(subset=["BOROUGH", "LATITUDE", "LONGITUDE"])
random_borough_entries = filtered_df[["BOROUGH", "LATITUDE", "LONGITUDE"]].sample(n=10)

print(random_borough_entries)

           BOROUGH   LATITUDE  LONGITUDE
1766217  MANHATTAN  40.747009 -73.977187
404769      QUEENS  40.748463 -73.827860
635351   MANHATTAN  40.762860 -73.989410
1094684      BRONX  40.847908 -73.903570
1086389   BROOKLYN  40.655773 -73.944410
754449       BRONX  40.842990 -73.826250
1312084   BROOKLYN  40.685324 -73.980703
956693      QUEENS  40.716625 -73.904625
1465161   BROOKLYN  40.654349 -73.921693
474486       BRONX  40.882633 -73.902190


In [27]:
def test_geocode_function(df):
    results = []
    for index, row in df.iterrows():
        lat = row['LATITUDE']
        lon = row['LONGITUDE']
        borough = geocode_borough(lat, lon)
        results.append(borough)
    return results
results = test_geocode_function(random_borough_entries)
actual_results = random_borough_entries[["BOROUGH"]]
comparison = pd.DataFrame({
    'Geocoded Borough': results,
    'Actual Borough': actual_results['BOROUGH']
})
print(comparison)

Location for (40.747009, -73.9771875): {'place_id': 331633083, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 2710839372, 'lat': '40.7467955', 'lon': '-73.9769299', 'class': 'place', 'type': 'house', 'place_rank': 30, 'importance': 9.175936522464359e-05, 'addresstype': 'place', 'name': '', 'display_name': '200, East 36th Street, Manhattan Community Board 6, Manhattan, New York County, New York, 10016, United States', 'address': {'house_number': '200', 'road': 'East 36th Street', 'neighbourhood': 'Manhattan Community Board 6', 'suburb': 'Manhattan', 'county': 'New York County', 'city': 'New York', 'state': 'New York', 'ISO3166-2-lvl4': 'US-NY', 'postcode': '10016', 'country': 'United States', 'country_code': 'us'}, 'boundingbox': ['40.7467455', '40.7468455', '-73.9769799', '-73.9768799']}
Location for (40.748463, -73.82786): {'place_id': 332100217, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/c